In [1]:
import pandas as pd
import os
import json
from datetime import datetime
import altair as alt
from tqdm.notebook import tqdm # Jupyter Notebook用の進捗バー

# AltairのグラフをNotebookに表示するための設定
alt.renderers.enable('default')

print("ライブラリのインポートが完了しました。")

ライブラリのインポートが完了しました。


In [2]:
# --- 定数定義 ---
INFLUENCERS_FILE = 'influencers.txt'
INFO_DIR = 'posts_info/unzipped_data_7z/info/'
OUTPUT_CSV_FILE = 'beauty_influencer_posts_summary.csv' # 出力ファイル名を変更

# --- 関数定義 ---

def load_influencer_data(filepath):
    """influencers.txtを読み込み,DataFrameを返す。"""
    try:
        df = pd.read_csv(filepath, sep='\t', skiprows=[1])
        return df
    except FileNotFoundError:
        print(f"エラー: `{filepath}` が見つかりません。")
        return pd.DataFrame()

def load_all_posts_data(influencer_list, info_dir):
    """
    指定されたインフルエンサーの全投稿データを読み込み,一つのDataFrameにまとめる。
    """
    all_posts = []
    
    print("指定されたインフルエンサーの投稿データを読み込んでいます...")
    for username in tqdm(influencer_list, desc="インフルエンサー"):
        try:
            all_files = os.listdir(info_dir)
            user_post_files = [f for f in all_files if f.startswith(f"{username}-") and f.endswith('.info')]

            for filename in user_post_files:
                filepath = os.path.join(info_dir, filename)
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                timestamp = data.get('taken_at_timestamp', 0)
                likes = data.get('edge_media_preview_like', {}).get('count', 0)
                comments = data.get('edge_media_to_parent_comment', {}).get('count', 0)

                all_posts.append({
                    'username': username,
                    'timestamp': timestamp,
                    'datetime': datetime.fromtimestamp(timestamp),
                    'likes': likes,
                    'comments': comments,
                })
        except FileNotFoundError:
            print(f"エラー: 投稿データディレクトリ `{info_dir}` が見つかりません。")
            return pd.DataFrame()
        except Exception as e:
            print(f"警告: {username}のデータ読み込み中にエラーが発生しました: {e}")
            
    if not all_posts:
        print("警告: 投稿データが一件も見つかりませんでした。")
        return pd.DataFrame()

    df = pd.DataFrame(all_posts)
    df = df.sort_values('datetime', ascending=False).reset_index(drop=True)
    return df

print("関数が定義されました。")

関数が定義されました。


In [3]:
# 1. 全インフルエンサーの情報をDataFrameとして読み込む
df_influencers = load_influencer_data(INFLUENCERS_FILE)

if not df_influencers.empty:
    # 2. Category列が 'beauty' のインフルエンサーに絞り込む (大文字・小文字を区別しない)
    df_beauty = df_influencers[df_influencers['Category'].str.lower() == 'beauty'].copy()
    
    if not df_beauty.empty:
        beauty_influencer_list = sorted(df_beauty['Username'].unique())
        print(f"「Beauty」カテゴリのインフルエンサーを {len(beauty_influencer_list)} 人見つけました。")
        print(beauty_influencer_list)
        
        # 3. 絞り込んだリストを元に,投稿データをDataFrameに集約
        df_all_posts = load_all_posts_data(beauty_influencer_list, INFO_DIR)
        
        if not df_all_posts.empty:
            print(f"\n処理が完了しました。合計 {len(df_all_posts)} 件の投稿データを読み込みました。")
            display(df_all_posts.head())
        else:
            print("データフレームが空です。")
    else:
        print("「Beauty」カテゴリのインフルエンサーが見つかりませんでした。")
else:
    print("インフルエンサー情報が取得できなかったため,処理を中断します。")

「Beauty」カテゴリのインフルエンサーを 1541 人見つけました。
['1208_sj', '15minbeauty', '1liz_vader3', '4everestherr', '54artistry', '99johannas', '_alussier', '_artacha', '_aubreanascurlss', '_beautykath_', '_bubsbee', '_chellita', '_chloemorton', '_geheimnisvoll', '_hannahweir_', '_kelsxo_', '_lahrin_', '_maryglow', '_micheleschmidt', '_sainte', '_sallyjo_', '_sugardenny_', '_taylor_bee_', '_tessfinley_', '_themakeupdoll', '_trishamarieditton_', '_wildfern', 'a1delatorre', 'aaqshah', 'abbeysoandso', 'abyxo', 'acoloradohairstylist', 'actually_ashly', 'adelina_ilardi_', 'adila_makeup', 'adrianaspink', 'adrianystefanyc', 'africanhairsummit', 'ahh_lexi', 'aidapersiaa', 'aimee.conroy', 'ainamarieh', 'airica.michelle', 'aisha.uk', 'alanaahewitt', 'alefabulash', 'alessaberry', 'alexander.danilov_', 'alexandralee1016', 'alexingamua', 'alexiscastro_xo', 'alexislives', 'alifeofclarity', 'alissa_adore', 'alissvlogchannel', 'alksne', 'alldaybunny', 'allieballiemakeup', 'allstuffnice', 'allthingsbeauty614', 'allureandmo

インフルエンサー:   0%|          | 0/1541 [00:00<?, ?it/s]

In [ ]:
if 'df_all_posts' in locals() and not df_all_posts.empty:
    # 保存する列を指定
    columns_to_save = ['datetime', 'username', 'likes', 'comments']
    df_all_posts[columns_to_save].to_csv(OUTPUT_CSV_FILE, index=False)
    print(f"全投稿データを '{OUTPUT_CSV_FILE}' として保存しました。")
else:
    print("保存するデータがありません。")

In [ ]:
if 'df_all_posts' in locals() and not df_all_posts.empty:
    # 対象インフルエンサーでデータをフィルタリング
    df_filtered = df_all_posts[df_all_posts['username'].isin(target_influencers)].copy()
    
    # 指定した期間で日付を丸める
    df_filtered['date'] = df_filtered['datetime'].dt.to_period(freq).dt.start_time

    # 投稿数を集計
    df_post_counts = df_filtered.groupby(['username', 'date']).size().reset_index(name='post_count')

    # Altairでグラフを作成
    chart = alt.Chart(df_post_counts, title=f"投稿数の推移 ({freq_map[freq]})").mark_line(point=True).encode(
        x=alt.X('date:T', title='日付'),
        y=alt.Y('post_count:Q', title='投稿数'),
        color=alt.Color('username:N', title='インフルエンサー'),
        tooltip=[
            alt.Tooltip('date:T', title='日付', format='%Y-%m-%d'),
            alt.Tooltip('username:N', title='名前'),
            alt.Tooltip('post_count:Q', title='投稿数')
        ]
    ).interactive()
    
    # グラフを表示
    display(chart)
else:
    print("グラフを描画するデータがありません。")